In [1]:
import os
import cv2
import numpy as np
import time
from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras.applications.vgg16 import preprocess_input as preprocess_vgg
from tensorflow.keras.applications.resnet50 import preprocess_input as preprocess_resnet
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')


## Chargement des données

In [2]:
def load_data(directory):
    data = []
    labels = []
    for label in ["R", "O"]:
        path = os.path.join(directory, label)
        for file in os.listdir(path):
            img_path = os.path.join(path, file)
            img = cv2.imread(img_path)
            if img is not None:
                img = cv2.resize(img, (128, 128))  # Taille arbitraire
                data.append(img)
                labels.append(0 if label == "R" else 1)  # 0 pour 'R', 1 pour 'O'
    return np.array(data), np.array(labels)

##  Détection de caractéristiques spatiales avec des réseaux de neurones profonds

Nous allons utiliser une architecture CNN comme ResNet et VGG pour extraire des caractéristiques de haut niveau des images. Par exemple, vous pouvez utiliser un modèle pré-entrainé comme ResNet50 de Keras et l'adapter pour votre tâche.

In [3]:
# Redimensionner les images à la taille appropriée
def resize_for_model(data, target_size=(224, 224)):
    resized_data = []
    for img in data:
        resized_img = cv2.resize(img, target_size)  # VGG16 et ResNet50 attendent 224x224
        resized_data.append(resized_img)
    return np.array(resized_data)

# Fonction pour extraire les caractéristiques avec ResNet50 ou VGG16
def extract_features(model, preprocess_func, data):
    data_preprocessed = preprocess_func(data)  # Prétraitement spécifique au modèle
    features = model.predict(data_preprocessed)  # Extraction des caractéristiques
    return features

In [4]:
# Préparer les données pour ResNet50 et VGG16
def prepare_data_for_models(X_train, X_test):
    print("Préparation des données...")
    X_train_resized = resize_for_model(X_train)
    X_test_resized = resize_for_model(X_test)

    # Charger les modèles pré-entraînés
    resnet_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')
    vgg_model = VGG16(weights='imagenet', include_top=False, pooling='avg')

    print("Extraction des caractéristiques avec ResNet50...")
    X_train_resnet = extract_features(resnet_model, preprocess_resnet, X_train_resized)
    X_test_resnet = extract_features(resnet_model, preprocess_resnet, X_test_resized)

    print("Extraction des caractéristiques avec VGG16...")
    X_train_vgg = extract_features(vgg_model, preprocess_vgg, X_train_resized)
    X_test_vgg = extract_features(vgg_model, preprocess_vgg, X_test_resized)

    # Retourner les caractéristiques extraites séparément pour chaque modèle
    return X_train_resnet, X_test_resnet, X_train_vgg, X_test_vgg

## ENTARINEMENT DES MODELES


In [5]:
def train_classifier(X_train_features, y_train, classifier_type='knn'):
    if classifier_type == 'knn':
        clf = KNeighborsClassifier(n_neighbors=5)
    elif classifier_type == 'rf':
        clf = RandomForestClassifier(n_estimators=100, random_state=42)
    elif classifier_type == 'xgb':
        clf = XGBClassifier(use_label_encoder=False, eval_metric="logloss")
    elif classifier_type == 'svm':
        clf = SVC(random_state=42)
    else:
        raise ValueError(f"Classificateur {classifier_type} non supporté")

    # Entraîner le modèle
    clf.fit(X_train_features, y_train)

    return clf




## EVALUATION DES MODELES


In [6]:

# Fonction d'évaluation du modèle
def evaluate_classifier(clf, X_test_features, y_test):

    start_time = time.time()

    # Prédiction des labels sur les données de test
    y_pred = clf.predict(X_test_features)

    # Calcul de la précision
    accuracy = accuracy_score(y_test, y_pred)

    # Durée de l'évaluation
    duration = time.time() - start_time

    return accuracy, duration


## Exécution du processus complet

In [7]:
# Chemins des répertoires d'entraînement et de test
train_dir = r"B:\computerVISIONFINALE\Train"
test_dir = r"B:\computerVISIONFINALE\Test"

# Charger les données
X_train, y_train = load_data(train_dir)
X_test, y_test = load_data(test_dir)

# Préparer les données pour ResNet50 et VGG16
X_train_resnet, X_test_resnet, X_train_vgg, X_test_vgg = prepare_data_for_models(X_train, X_test)

# Entraîner et évaluer les modèles pour ResNet50
clf_knn_resnet = train_classifier(X_train_resnet, y_train, classifier_type='knn')
clf_svm_resnet = train_classifier(X_train_resnet, y_train, classifier_type='svm')
clf_xgb_resnet = train_classifier(X_train_resnet, y_train, classifier_type='xgb')
clf_rf_resnet = train_classifier(X_train_resnet, y_train, classifier_type='rf')

accuracy_knn_resnet, duration_knn_resnet = evaluate_classifier(clf_knn_resnet, X_test_resnet, y_test)
accuracy_svm_resnet, duration_svm_resnet = evaluate_classifier(clf_svm_resnet, X_test_resnet, y_test)
accuracy_xgb_resnet, duration_xgb_resnet = evaluate_classifier(clf_xgb_resnet, X_test_resnet, y_test)
accuracy_rf_resnet, duration_rf_resnet = evaluate_classifier(clf_rf_resnet, X_test_resnet, y_test)

# Entraîner et évaluer les modèles pour VGG16
clf_knn_vgg = train_classifier(X_train_vgg, y_train, classifier_type='knn')
clf_svm_vgg = train_classifier(X_train_vgg, y_train, classifier_type='svm')
clf_xgb_vgg = train_classifier(X_train_vgg, y_train, classifier_type='xgb')
clf_rf_vgg = train_classifier(X_train_vgg, y_train, classifier_type='rf')

accuracy_knn_vgg, duration_knn_vgg = evaluate_classifier(clf_knn_vgg, X_test_vgg, y_test)
accuracy_svm_vgg, duration_svm_vgg = evaluate_classifier(clf_svm_vgg, X_test_vgg, y_test)
accuracy_xgb_vgg, duration_xgb_vgg = evaluate_classifier(clf_xgb_vgg, X_test_vgg, y_test)
accuracy_rf_vgg, duration_rf_vgg = evaluate_classifier(clf_rf_vgg, X_test_vgg, y_test)

# Afficher les résultats pour ResNet50 et VGG16
print("Résultats pour ResNet50 :")
print(f"Précision KNN : {accuracy_knn_resnet:.4f}, Temps : {duration_knn_resnet:.4f}")
print(f"Précision SVM : {accuracy_svm_resnet:.4f}, Temps : {duration_svm_resnet:.4f}")
print(f"Précision XGBoost : {accuracy_xgb_resnet:.4f}, Temps : {duration_xgb_resnet:.4f}")
print(f"Précision RandomForest : {accuracy_rf_resnet:.4f}, Temps : {duration_rf_resnet:.4f}")

print("\nRésultats pour VGG16 :")
print(f"Précision KNN : {accuracy_knn_vgg:.4f}, Temps : {duration_knn_vgg:.4f}")
print(f"Précision SVM : {accuracy_svm_vgg:.4f}, Temps : {duration_svm_vgg:.4f}")
print(f"Précision XGBoost : {accuracy_xgb_vgg:.4f}, Temps : {duration_xgb_vgg:.4f}")
print(f"Précision RandomForest : {accuracy_rf_vgg:.4f}, Temps : {duration_rf_vgg}")


Préparation des données...
Extraction des caractéristiques avec ResNet50...
370/370 ━━━━━━━━━━━━━━━━━━━━ 439s 1s/step
159/159 ━━━━━━━━━━━━━━━━━━━━ 176s 1s/step
Extraction des caractéristiques avec VGG16...
370/370 ━━━━━━━━━━━━━━━━━━━━ 1088s 3s/step
159/159 ━━━━━━━━━━━━━━━━━━━━ 473s 3s/step
Résultats pour ResNet50 :
Précision KNN : 0.9570, Temps : 2.6875
Précision SVM : 0.9702, Temps : 18.6723
Précision XGBoost : 0.9568, Temps : 0.0444
Précision RandomForest : 0.9435, Temps : 0.1884

Résultats pour VGG16 :
Précision KNN : 0.9374, Temps : 0.5265
Précision SVM : 0.9550, Temps : 5.7880
Précision XGBoost : 0.9453, Temps : 0.0147
Précision RandomForest : 0.9238, Temps : 0.11669182777404785


In [16]:
from sklearn.metrics import classification_report

# Afficher les résultats pour ResNet50
print("Résultats pour ResNet50 :")
# Résultats pour KNN
print("\nKNN - ResNet50:")
print(f"Précision : {accuracy_knn_resnet:.4f}, Temps : {duration_knn_resnet:.4f}")
y_pred_knn_resnet = clf_knn_resnet.predict(X_test_resnet)
print(classification_report(y_test, y_pred_knn_resnet))

# Résultats pour SVM
print("\nSVM - ResNet50:")
print(f"Précision : {accuracy_svm_resnet:.4f}, Temps : {duration_svm_resnet:.4f}")
y_pred_svm_resnet = clf_svm_resnet.predict(X_test_resnet)
print(classification_report(y_test, y_pred_svm_resnet))

# Résultats pour XGBoost
print("\nXGBoost - ResNet50:")
print(f"Précision : {accuracy_xgb_resnet:.4f}, Temps : {duration_xgb_resnet:.4f}")
y_pred_xgb_resnet = clf_xgb_resnet.predict(X_test_resnet)
print(classification_report(y_test, y_pred_xgb_resnet))

# Résultats pour RandomForest
print("\nRandomForest - ResNet50:")
print(f"Précision : {accuracy_rf_resnet:.4f}, Temps : {duration_rf_resnet:.4f}")
y_pred_rf_resnet = clf_rf_resnet.predict(X_test_resnet)
print(classification_report(y_test, y_pred_rf_resnet))

# Afficher les résultats pour VGG16
print("\nRésultats pour VGG16 :")
# Résultats pour KNN
print("\nKNN - VGG16:")
print(f"Précision : {accuracy_knn_vgg:.4f}, Temps : {duration_knn_vgg:.4f}")
y_pred_knn_vgg = clf_knn_vgg.predict(X_test_vgg)
print(classification_report(y_test, y_pred_knn_vgg))

# Résultats pour SVM
print("\nSVM - VGG16:")
print(f"Précision : {accuracy_svm_vgg:.4f}, Temps : {duration_svm_vgg:.4f}")
y_pred_svm_vgg = clf_svm_vgg.predict(X_test_vgg)
print(classification_report(y_test, y_pred_svm_vgg))

# Résultats pour XGBoost
print("\nXGBoost - VGG16:")
print(f"Précision : {accuracy_xgb_vgg:.4f}, Temps : {duration_xgb_vgg:.4f}")
y_pred_xgb_vgg = clf_xgb_vgg.predict(X_test_vgg)
print(classification_report(y_test, y_pred_xgb_vgg))

# Résultats pour RandomForest
print("\nRandomForest - VGG16:")
print(f"Précision : {accuracy_rf_vgg:.4f}, Temps : {duration_rf_vgg:.4f}")
y_pred_rf_vgg = clf_rf_vgg.predict(X_test_vgg)
print(classification_report(y_test, y_pred_rf_vgg))


Résultats pour ResNet50 :

KNN - ResNet50:
Précision : 0.9570, Temps : 2.6875
              precision    recall  f1-score   support

           0       0.96      0.94      0.95      2133
           1       0.95      0.97      0.96      2931

    accuracy                           0.96      5064
   macro avg       0.96      0.95      0.96      5064
weighted avg       0.96      0.96      0.96      5064


SVM - ResNet50:
Précision : 0.9702, Temps : 18.6723
              precision    recall  f1-score   support

           0       0.97      0.96      0.96      2133
           1       0.97      0.98      0.97      2931

    accuracy                           0.97      5064
   macro avg       0.97      0.97      0.97      5064
weighted avg       0.97      0.97      0.97      5064


XGBoost - ResNet50:
Précision : 0.9568, Temps : 0.0444
              precision    recall  f1-score   support

           0       0.96      0.93      0.95      2133
           1       0.95      0.97      0.96      2

## Save the best Model

In [8]:
import joblib

# Comparer les précisions pour chaque modèle
best_accuracy = 0
best_model = None
best_classifier_type = ""
best_model_name = ""

# ResNet50
if accuracy_knn_resnet > best_accuracy:
    best_accuracy = accuracy_knn_resnet
    best_model = clf_knn_resnet
    best_classifier_type = 'knn'
    best_model_name = 'best_knn_resnet.pkl'

if accuracy_svm_resnet > best_accuracy:
    best_accuracy = accuracy_svm_resnet
    best_model = clf_svm_resnet
    best_classifier_type = 'svm'
    best_model_name = 'best_svm_resnet.pkl'

if accuracy_xgb_resnet > best_accuracy:
    best_accuracy = accuracy_xgb_resnet
    best_model = clf_xgb_resnet
    best_classifier_type = 'xgb'
    best_model_name = 'best_xgb_resnet.pkl'

if accuracy_rf_resnet > best_accuracy:
    best_accuracy = accuracy_rf_resnet
    best_model = clf_rf_resnet
    best_classifier_type = 'rf'
    best_model_name = 'best_rf_resnet.pkl'

# VGG16
if accuracy_knn_vgg > best_accuracy:
    best_accuracy = accuracy_knn_vgg
    best_model = clf_knn_vgg
    best_classifier_type = 'knn'
    best_model_name = 'best_knn_vgg.pkl'

if accuracy_svm_vgg > best_accuracy:
    best_accuracy = accuracy_svm_vgg
    best_model = clf_svm_vgg
    best_classifier_type = 'svm'
    best_model_name = 'best_svm_vgg.pkl'

if accuracy_xgb_vgg > best_accuracy:
    best_accuracy = accuracy_xgb_vgg
    best_model = clf_xgb_vgg
    best_classifier_type = 'xgb'
    best_model_name = 'best_xgb_vgg.pkl'

if accuracy_rf_vgg > best_accuracy:
    best_accuracy = accuracy_rf_vgg
    best_model = clf_rf_vgg
    best_classifier_type = 'rf'
    best_model_name = 'best_rf_vgg.pkl'

# Enregistrer le meilleur modèle
if best_model is not None:
    joblib.dump(best_model, best_model_name)
    print(f"Le meilleur modèle est {best_classifier_type} avec une précision de {best_accuracy:.4f}.")
    print(f"Le modèle a été enregistré sous le nom : {best_model_name}")
else:
    print("Aucun modèle n'a été trouvé.")


Le meilleur modèle est svm avec une précision de 0.9702.
Le modèle a été enregistré sous le nom : best_svm_resnet.pkl


# Conclusion 
La classification des images a montré une amélioration significative par rapport à la première partie, grâce à l'utilisation de caractéristiques de haut niveau extraites via un réseau de neurones profond.

**Meilleurs Résultats - Partie 1**

<img src="score_partie1.png" alt="Résultats Partie 1" width="500" height="200" />


**Impact de l'utilisation de ResNet50**

L'intégration de caractéristiques de haut niveau, obtenues grâce au réseau de neurones ResNet50, a permis de considérablement affiner les performances du modèle. En particulier :

* Amélioration des métriques globales : Les scores de précision, de rappel et de F1-score ont enregistré une nette progression.

* Classification plus précise : Le modèle est désormais plus efficace pour différencier les classes "Recyclable" et "Organique", offrant ainsi une meilleure fiabilité.

Cependant, cette amélioration s'accompagne d'une augmentation du temps d'exécution, mettant en évidence la nécessité de compromis entre précision et rapidité selon les exigences du contexte applicatif.